In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Yola_fit').getOrCreate()



In [2]:
df = spark.read.csv('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Pricing_fallback_rule.csv', header='True')



In [3]:
df.createOrReplaceTempView('Fallback_rule')

In [4]:
df.show()

+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|         item_code|      date|item_gtin_comparable|  item_main_category| item_sub_category_1|item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin|nov|shop_gross_margin|shop_pc2_margin|
+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|000000001000052666|2020-02-16|          comparab

In [5]:
import pandas as pd
import numpy as np
import patsy as pat



In [6]:
pdf_sap = pd.read_excel('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Lieferanten_Master_Data_SAP_V2.xlsx',header=3, usecols=['SAP_ID','Supplier_Name','Type'])#,'Type'])
pdf_sap.dropna(inplace=True)
print(pdf_sap.head(100))


#df_sap = spark.createDataFrame(pdf_sap)



       SAP_ID                         Supplier_Name         Type
2    100000.0                    Art of Living GmbH  Marketplace
3    100001.0            Actona Company AS (Inland)     Domestic
4    100002.0   Actona Company A/S (Hauptwerk SHANG   FOB Import
5    100003.0                      Actual Diffusion  Marketplace
6    100004.0                   Arte M GmbH & Co.KG     Domestic
..        ...                                   ...          ...
103  100109.0   Duo Collection Import Vertriebsgese     Domestic
104  100110.0                          David & Luna     Domestic
105  100111.0  design3000 Vertriebsgesellschaft mbH  Marketplace
106  100112.0             Dreisitz-design Sp.z.o.o.     Domestic
107  100113.0                          Dan-Form A/S     Domestic

[100 rows x 3 columns]


In [7]:
print(pdf_sap.dtypes)
pdf_sap_filt = pdf_sap.astype({'SAP_ID': 'int32', 'Supplier_Name': 'string','Type':'string'})
print(pdf_sap_filt.dtypes)



SAP_ID           float64
Supplier_Name     object
Type              object
dtype: object
SAP_ID            int32
Supplier_Name    string
Type             string
dtype: object


In [8]:
df_sap = spark.createDataFrame(pdf_sap_filt)
df_sap.createOrReplaceTempView('SAP_id')

In [9]:
df_sap_INJOIN = spark.sql("""SELECT fbr.*, sap.Type
                                from Fallback_rule as fbr
                              INNER JOIN SAP_id as sap on sap.SAP_ID = CAST(fbr.item_supplier_regular_id as INT) """)

In [10]:
df_sap_INJOIN.createOrReplaceTempView('IN_JOIN_Table')

In [11]:
df_sap_INJOIN.show()

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|000000001000097652|20

In [12]:
df_fin_table = spark.sql(""" SELECT ijt.*,
        CASE
           when ijt.item_inco_terms = 'FOB'
               then 'Import'
           when LOWER(ijt.Type) LIKE 'eastern europe'
               then 'EE'
           when LOWER(ijt.Type) LIKE 'import'
               then 'Import'
           else 'Domestic' END                                                              as sourcing_location
            from IN_JOIN_Table as ijt
""")
#shop_pc2_margin/NOV


In [13]:
df_fin_table.show()
df_fin_table.createOrReplaceTempView('Fin_table')

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-----------------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|sourcing_location|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+------------

In [14]:
df_test = spark.sql("""SELECT DISTINCT item_main_category from Fin_table ORDER BY item_main_category ASC""")

In [15]:
df_test.show()

+--------------------+
|  item_main_category|
+--------------------+
|                BABY|
|                 BAD|
|              BETTEN|
|            BOUTIQUE|
|                BÜRO|
|ESSZIMMERSTÜHLE&B...|
|     ESSZIMMERTISCHE|
|           FLURMÖBEL|
|              GARTEN|
|            HAUSHALT|
|       HEIMTEXTILIEN|
|              KINDER|
|               KÜCHE|
|            LEUCHTEN|
|MATRATZEN&LATTENR...|
|             POLSTER|
|   SCHLAFZIMMERMÖBEL|
|TEPPICHE & BODENB...|
|              WOHNEN|
+--------------------+



In [16]:
df_item_dates = spark.sql("""SELECT * from Fin_table where item_code = '000000001000097478' ORDER BY date ASC""")
df_item_dates.show(500)

+------------------+----------+--------------------+------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-----------------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1|item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|sourcing_location|
+------------------+----------+--------------------+------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------

In [17]:
pdf_fin = df_fin_table.toPandas()



In [18]:
#MODEL IMPLEMENTATION
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

list_of_num_columns = ['wh_flag','shop_pc2_margin','pocket_pc2_margin','nov','shop_gross_margin']
for col in list_of_num_columns:
    pdf_fin[col]=pd.to_numeric(pdf_fin[col])
#pdf_fin = pdf_fin.astype({'wh_flag':'int32', 'shop_pc2_margin':'int32','item_main_category':'string','exclusivity_clean':'string','sourcing_location':'string' })


In [19]:
print(pdf_fin.dtypes)

item_code                   object
date                        object
item_gtin_comparable        object
item_main_category          object
item_sub_category_1         object
item_sub_category_2         object
item_supplier_regular_id    object
item_inco_terms             object
exclusivity_clean           object
item_supplier_regular       object
inventory_flag              object
wh_flag                      int64
item_final_price_rule       object
sellability_laso_de         object
pocket_pc2_margin            int64
nov                          int64
shop_gross_margin            int64
shop_pc2_margin              int64
Type                        object
sourcing_location           object
dtype: object


In [20]:
desc = pat.ModelDesc.from_formula("shop_pc2_margin ~  C(item_main_category) + C(exclusivity_clean) + C(sourcing_location) + C(wh_flag) -1 ")
desc.describe()

'shop_pc2_margin ~ 0 + C(item_main_category) + C(exclusivity_clean) + C(sourcing_location) + C(wh_flag)'

In [21]:
reg_model_with_fe = smf.ols("shop_pc2_margin ~  C(item_main_category) + C(exclusivity_clean) + C(sourcing_location) + C(wh_flag) ",  pdf_fin,hasconst = True).fit()
print(reg_model_with_fe.summary())
#



                            OLS Regression Results                            
Dep. Variable:        shop_pc2_margin   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     6161.
Date:                Fri, 29 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:55:56   Log-Likelihood:            -4.5319e+06
No. Observations:              713360   AIC:                         9.064e+06
Df Residuals:                  713334   BIC:                         9.064e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [22]:
WH_with_fe = smf.ols("shop_pc2_margin ~  C(wh_flag) ",  pdf_fin,hasconst = True).fit()
print(WH_with_fe.summary())
#


                            OLS Regression Results                            
Dep. Variable:        shop_pc2_margin   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                 1.798e+04
Date:                Fri, 29 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:55:59   Log-Likelihood:            -4.5928e+06
No. Observations:              713360   AIC:                         9.186e+06
Df Residuals:                  713358   BIC:                         9.186e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          84.4335      0.314    2

In [24]:
WH_with_fe_false = smf.ols("shop_pc2_margin ~  C(wh_flag) + C(sourcing_location) + C(exclusivity_clean) +C(item_main_category) -1",  pdf_fin,hasconst = False).fit()
print(WH_with_fe_false.summary())
#


                                 OLS Regression Results                                
Dep. Variable:        shop_pc2_margin   R-squared (uncentered):                   0.487
Model:                            OLS   Adj. R-squared (uncentered):              0.487
Method:                 Least Squares   F-statistic:                          2.607e+04
Date:                Fri, 29 Jan 2021   Prob (F-statistic):                        0.00
Time:                        15:58:17   Log-Likelihood:                     -4.5319e+06
No. Observations:              713360   AIC:                                  9.064e+06
Df Residuals:                  713334   BIC:                                  9.064e+06
Df Model:                          26                                                  
Covariance Type:            nonrobust                                                  
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----

In [ ]:
print('Old')


In [ ]:
WH_with_fe_false = smf.ols("shop_pc2_margin/nov ~  C(wh_flag) + C(sourcing_location) ",  pdf_fin,hasconst = False).fit()
print(WH_with_fe_false.summary())
#



In [ ]:
WH_with_fe_false = smf.ols("shop_pc2_margin ~  C(wh_flag)*C(sourcing_location)  -1",  pdf_fin,hasconst = False).fit()
print(WH_with_fe_false.summary())
#



In [ ]:
pdf_fin_dom = pdf_fin[pdf_fin['sourcing_location']== 'Domestic']

reg_model_with_fe_dom = smf.ols("shop_pc2_margin ~ pocket_pc2_margin + C(wh_flag) + C(item_main_category) + C(exclusivity_clean) + C(sourcing_location)", pdf_fin_dom).fit()
print(reg_model_with_fe_dom.summary())
